In [2]:
from MFEA_lib.tasks.Benchmark.IDPC_EDU import IDPC_EDU_benchmark
from MFEA_lib.model import MFEA_base
from MFEA_lib.model.utils import *
from MFEA_lib.operators.Crossover import *
from MFEA_lib.operators.Mutation import *
from MFEA_lib.operators.Selection import *

In [3]:
tasks, IndClass = IDPC_EDU_benchmark.get_tasks(1)

  0%|          | 0/24 [00:00<?, ?it/s]

Reading data...


100%|██████████| 24/24 [00:07<00:00,  3.25it/s]


In [8]:
tasks[0].num_nodes

45

In [4]:
baseModel = MFEA_base.model()
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    # crossover= newSBX(nc = 2, gamma= 0.4, alpha= 6),
    crossover= NoCrossover(nc = 2),
    mutation= IDPCEDU_Mutation(nm = 5),
    selection= ElitismSelection()
)
solve = baseModel.fit(
    nb_generations = 1000, rmp = 0.3, nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

IndexError: index 14 is out of bounds for axis 0 with size 10